In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import numba
import grid2op
import numpy as np
from lightsim2grid import LightSimBackend           # highly recommended!
from MyRewards import RLReward

In [ ]:
lambdaa = 0.03
line_switch_cost = 0.5

selected_reward_function = RLReward(lambdaa, line_switch_cost)
#selected_action_cost = ActionCost(lambdaa, line_switch_cost)
#other_rewards = { "margin_reward": MarginReward, "action_cost": selected_action_cost }

env_name = "l2rpn_neurips_2020_track1_small_train_random_32"

env = grid2op.make(env_name,
                   reward_class=selected_reward_function,
                   #param=custom_params,
                   backend=LightSimBackend(),
                   opponent_init_budget=0, opponent_budget_per_ts=0 # removing the influence of attacks
                   )

In [ ]:
def powerFlowLimitsFunction(env):
    """
    Power Flow Limits array (necessary, needed only once at the start) 
    Size: number of branches in the "initial/starting" grid.
    power-flow limits on each line in the "INITIAL" power-grid. Go to array to extract the power-line limit.
    after my thorough research, power flow limits on each grid2op branch (transmission line and transformer) can be found as:
    """
    baseVoltagesForEachLine = np.array([138]*45 + [345]*3 + [138]*7 + [345]*4)         # manually building it since the base voltages can be recognized from the previous section.
    powerFlowLimits = np.abs(env.get_thermal_limit())*baseVoltagesForEachLine*1000*np.sqrt(3)
    powerFlowLimits = powerFlowLimits/1e6                          # power flow limits in MWs
    return powerFlowLimits

powerFlowLimits = powerFlowLimitsFunction(env) # will not change with time steps
init_obs = env.reset()

from DoubleDuelingDQN_Redis_LODF import DoubleDuelingDQN as BackupAgent

agent = BackupAgent(env.observation_space, env.action_space, env._init_grid_path, init_obs, powerFlowLimits, name = "DDDQN_random_train", is_training = True)

In [ ]:
""" 
# in case you want to use a pre-trained agent
import os
submission_dir = "C:\\Users\\dwivea2\\Desktop\\RL code\\CustomAgent _Redispatch+LODF"
agent.load(os.path.join(submission_dir, "75000_36bus_posttrain-model_anmol_redis_LODF-try5", "DDDQN_random_train.h5"))
"""

In [ ]:
agent.train(env,
            10e4,  # num of steps to train
            save_path = "36bus_posttrain-model_anmol_redis_LODF-try12" + "_" + str(line_switch_cost) + "_" + str(lambdaa),
            activateAgentRho = 0.95,
            logdir = "logs-random-train-anmol-redis_LODF-try12" + "_" + str(line_switch_cost) + "_" + str(lambdaa)
           )